#  run the lfnet

In [1]:
import cv2
import os
import tqdm
import torch
import random
import argparse
import numpy as np

from utils.common_utils import gct
from utils.eval_utils import nearest_neighbor_distance_ratio_match
from model.rf_des import HardNetNeiMask
from model.rf_det_so import RFDetSO
from model.rf_net_so import RFNetSO
from config import cfg

In [2]:
print(f"{gct()} : start time")

random.seed(cfg.PROJ.SEED)
torch.manual_seed(cfg.PROJ.SEED)
np.random.seed(cfg.PROJ.SEED)

print(f"{gct()} : model init")
det = RFDetSO(
    cfg.TRAIN.score_com_strength,
    cfg.TRAIN.scale_com_strength,
    cfg.TRAIN.NMS_THRESH,
    cfg.TRAIN.NMS_KSIZE,
    cfg.TRAIN.TOPK,
    cfg.MODEL.GAUSSIAN_KSIZE,
    cfg.MODEL.GAUSSIAN_SIGMA,
    cfg.MODEL.KSIZE,
    cfg.MODEL.padding,
    cfg.MODEL.dilation,
    cfg.MODEL.scale_list,
)
des = HardNetNeiMask(cfg.HARDNET.MARGIN, cfg.MODEL.COO_THRSH)
model = RFNetSO(
    det, des, cfg.LOSS.SCORE, cfg.LOSS.PAIR, cfg.PATCH.SIZE, cfg.TRAIN.TOPK
)

print(f"{gct()} : to device")
device = torch.device("cuda")
model = model.to(device)
resume = "../runs/10_24_09_25/model/e121_NN_0.480_NNT_0.655_NNDR_0.813_MeanMS_0.649.pth.tar"
print(f"{gct()} : in {resume}")
checkpoint = torch.load(resume)
model.load_state_dict(checkpoint["state_dict"])

08-17 14:32:11 : start time
08-17 14:32:11 : model init
08-17 14:32:11 : to device
08-17 14:32:13 : in ../runs/10_24_09_25/model/e121_NN_0.480_NNT_0.655_NNDR_0.813_MeanMS_0.649.pth.tar


In [4]:
class arg:
    def __init__(self):
        self.in_dir = '/home/wang/workspace/data/rgbd_dataset_freiburg1_room/rgb'
        self.out_dir = '/home/wang/workspace/data/rgbd_dataset_freiburg1_room/RFNet_DBOW/'
        self.max_longer_edge = 640
        
config = arg()

# Ready to feed input images

In [5]:
img_paths = [x.path for x in os.scandir(config.in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
print('Found {} images...'.format(len(img_paths)))

if not os.path.exists(config.out_dir):
    os.makedirs(config.out_dir)

avg_elapsed_time = 0

for img_path in tqdm.tqdm(img_paths):
    photo = cv2.imread(img_path)
    height, width = photo.shape[:2]
    # Dump keypoint locations and their features
    kp, des, _ ,scale,ori= model.detectAndCompute(img_path, device, (height, width))
    out_kpts_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts.txt')
    out_feats_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_feats.txt')
    out_kpts_ori_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts_ori.txt')
    out_kpts_scale_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts_scale.txt')
    
    np.savetxt(out_kpts_path, kp[:,1:3].cpu().detach().numpy())
    np.savetxt(out_feats_path, des.cpu().detach().numpy())
    np.savetxt(out_kpts_ori_path, np.degrees(np.arctan((ori[:,1]/ori[:,0]).cpu().detach().numpy())))
    np.savetxt(out_kpts_scale_path, scale.cpu().detach().numpy())
    #np.savez(out_path, kpts=outs['kpts'], feats=outs['feats'], size=np.array([height, width]))
print('Done.')

  0%|          | 0/1362 [00:00<?, ?it/s]

Found 1362 images...


/home/wang/workspace/RFNET/rfnet/model/rf_det_module.py:202: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  None, None, :, :
100%|██████████| 1362/1362 [07:24<00:00,  3.04it/s]

Done.
